# Walkthrough - Estimate calories available country-wide

Warning: I am not an expert! This is just a rough 'back of the napkin' calculation. The idea is to see if we can calculate approximately how many calories (kcal) of food are available in Sudan between September 2023 and September 2024. The approach is based on my best reconstruction from what I read in the Clingendael reports and some limited personal correspondence with someone in the field. If you find problems or have suggestions for improvement, please let me know!

## Sources

- [Sudan: From hunger to death](https://www.clingendael.org/publication/sudan-hunger-death) 
2024-05-24, Dr. Timmo Gaasbeek. 

- [From Catastrophe to Famine: Immediate action needed in Sudan to contain mass starvation](https://www.clingendael.org/publication/catastrophe-famine-immediate-action-needed-sudan-contain-mass-starvation) 2024-02-08. Anette Hoffmann. 

For information on data sources, see /data/metadata/sources.md

## Process overview

1. Load data on [current grain harvest](##-Load-data-on-harvest)
2. Add estimates of [other grain sources](##-Add-estimates-for-other-grain-sources)
3. Convert [from metric tons of grain to calories](##-Convert-from-metric-tons-of-grain-to-calories)
4. Add in [non-grain sources of food](##-Include-non-grain-sources-of-food)
5. Sanity check - assuming equal distribution, [how many days of food is that for the full population?](##-sanity-check)



In [24]:
import os
import re
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

try:
    base_path = Path(__file__).resolve().parent.parent
except NameError:
    base_path = Path.cwd().parent.parent 

In [ ]:
print(base_path)

## Load data on harvest

[CFSAM national cereal production estimate](https://www.fao.org/markets-and-trade/publications/detail/en/c/1679419/)
citation in Clingendael report as "FAO, 2024. Special Report 2023: FAO Crop and Food Supply Assessment Mission (CFSAM) to the Sudan, 19 March 2024"


[NB: This matches the summary reported available here](https://reliefweb.int/report/sudan/special-report-2023-fao-crop-and-food-supply-assessment-mission-cfsam-sudan-19-march-2024-enar): "National cereal production in 2023, including wheat crops to be harvested in March 2024, is estimated at about 4.1 million tonnes"

NB: "two-thirds of the grain consumed in Sudan is grown locally: mostly sorghum and millet, and some wheat." (Clingendael, From Catastrophe to Famine, p.3)


In [25]:
cereal_file = base_path / 'data' / 'raw' / 'available_calories' / 'FAO_CFSAM_cereal_production.csv'

cerdf = pd.read_csv(cereal_file)
cerdf.columns = cerdf.columns.str.replace('/', '_')
cerdf.columns = cerdf.columns.str.replace(' ', '_')

# just get totals - 
totcereals = cerdf[cerdf.State_Scheme_Sector=='Grand total']

tot_sorg = float(totcereals['Sorghum_-_2023_24'].values[0]) * 1000
tot_millet = float(totcereals['Millet_-_2023_24'].values[0])* 1000
tot_wheat = float(totcereals['Wheat_-_2023_24'].values[0])* 1000


total_domestic = tot_sorg + tot_millet + tot_wheat
print(f"domestically-produced grains 2023-24 (est): {total_domestic:.1f} MT")


domestically-produced grains 2023-24 (est): 4116230.0 MT


## Add estimates for other grain sources

(details forthcoming)


In [26]:
# Cereal availability in metric tons (MT)
household_stocks = 320000  # Household-level stocks in MT
shop_stocks = 50000  # Stocks in smaller shops in MT
company_imports = 800000  # Imports by large companies in MT
informal_imports = 300000  # Informal imports in MT
food_aid_imports = 150000  # Food aid imports in MT
wild_grains = 250000  # Wild grains in MT


We don't know the composition of these different stocks in terms of wheat vs millet vs sorghum, etc. 
GPT-4 thinks it can guess, and it's probably wrong, but probably not much worse than me, so I'll go with that. For example, it does make sense that there would be more wild sorghum and millet than wild wheat. In any case, as you'll see below, the differences are relatively minor in terms of caloric content.

In [27]:
# Assumed composition percentages for each food source category

# Household stocks: 50% sorghum, 30% millet, 20% wheat
household_sorghum = household_stocks * 0.50
household_millet = household_stocks * 0.30
household_wheat = household_stocks * 0.20

# Shop stocks: 40% wheat, 30% sorghum, 30% millet
shop_sorghum = shop_stocks * 0.30
shop_millet = shop_stocks * 0.30
shop_wheat = shop_stocks * 0.40

# Company imports: 80% wheat, 10% sorghum, 10% millet
company_sorghum = company_imports * 0.10
company_millet = company_imports * 0.10
company_wheat = company_imports * 0.80

# Informal imports: 60% wheat, 20% sorghum, 20% millet
informal_sorghum = informal_imports * 0.20
informal_millet = informal_imports * 0.20
informal_wheat = informal_imports * 0.60

# Food aid imports: 70% wheat, 20% sorghum, 10% millet
food_aid_sorghum = food_aid_imports * 0.20
food_aid_millet = food_aid_imports * 0.10
food_aid_wheat = food_aid_imports * 0.70

# Wild grains: 60% sorghum, 40% millet
wild_sorghum = wild_grains * 0.60
wild_millet = wild_grains * 0.40
wild_wheat = 0  # No wheat expected in wild grains

total_other_sorghum = household_sorghum + shop_sorghum + company_sorghum + informal_sorghum + food_aid_sorghum + wild_sorghum
total_other_millet = household_millet + shop_millet + company_millet + informal_millet + food_aid_millet + wild_millet
total_other_wheat = household_wheat + shop_wheat + company_wheat + informal_wheat + food_aid_wheat + wild_wheat

print(f"Other sorghum: {total_other_sorghum} metric tons")
print(f"Other millet: {total_other_millet} metric tons")
print(f"Other wheat: {total_other_wheat} metric tons")

Other sorghum: 495000.0 metric tons
Other millet: 366000.0 metric tons
Other wheat: 1009000.0 metric tons


In [28]:
# Now, combine this with the harvest data
ALL_sorg = tot_sorg + total_other_sorghum
ALL_millet = tot_millet + total_other_millet
ALL_wheat = tot_wheat + total_other_wheat

### Assume some stocks remain
It is unlikely that every kilo of grain is consumed at the end of the time period. Presumably some will remain in stocks for various reasons. Assume that roughly 20 metric tons will remain unconsumed.

In [29]:
ALL_sorg -= 6.6
ALL_millet -= 6.6
ALL_wheat -= 6.6

## Convert from metric tons of grain to calories

"Comparative Study of Nutritional Value of Wheat, Maize, Sorghum, Millet, and Fonio: Some Cereals Commonly Consumed in Côte d’Ivoire" European scientific Journal
https://eujournal.org/index.php/esj/article/view/13166/13300

In [30]:

kcal_conv_file = base_path / 'data' / 'raw' / 'available_calories' / 'Jocelyne_etal_nutritional_estimates.csv'

kcal  = pd.read_csv(kcal_conv_file)

kcal[kcal.parameter == "EV_Kcal_per_100g_DM"]

wheat_conv =  kcal[kcal.parameter == "EV_Kcal_per_100g_DM"]['Wheat'].values[0]
sorg_conv =  kcal[kcal.parameter == "EV_Kcal_per_100g_DM"]['Sorghum'].values[0]
millet_conv =  kcal[kcal.parameter == "EV_Kcal_per_100g_DM"]['Millet'].values[0]

print(f"wheat conv: {wheat_conv} Kcal_per_100g_DM")
print(f"sorghum conv: {sorg_conv} Kcal_per_100g_DM")
print(f"millet conv: {millet_conv} Kcal_per_100g_DMs")


wheat conv: 308.22 Kcal_per_100g_DM
sorghum conv: 308.84 Kcal_per_100g_DM
millet conv: 319.39 Kcal_per_100g_DMs



The above conversion (EV (Energy Value) (Kcal / 100g DM)) is based on 100g dry weight of grain. So we need to take out the part of th weight that is water, and convert the kilos into 100-gram portions (the EV data is per 100 grams).

I looked up some data on moisture content, but it is fairly complicated as it depends on local conditions, time after harvest, etc. so for now I just say 11% for all. For details, see the following sources:

- [This source on millet](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6582004/) suggests percent moisture of 10.45-11.02%.

 <!-- "Initial moisture content of raw grains was determined for the triplicate samples by measuring the loss in weight of 5 g sample by drying at 130 ± 2 °C for 2 h (AACC2003) in hot air oven. 
"The initial moisture content and equilibrium moisture content of the little millet grain were 11.02% (d.b.) and 38–49.8% (d.b.), respectively"
Proso millet grain ... initial moisture content of 10.45% (db.) -->


- [Wheat percent moisture in Iran](https://www.sciencedirect.com/science/article/pii/S2214317319301696)
<!--  Prediction of wheat moisture content at harvest time through ANN and SVR modeling techniques
Iran - 15% - cereals in 2014/15. ... "the highest agricultural production rate of 14.9% was attributed to wheat in this crop year
Wheat harvesting operation starts when the crop is at physiological maturity and its moisture content falls to 14%, but it could be varied according to management policies.
"In Iran, because of the sufficiently high air temperatures and the lack of heavy rainfalls or extreme weather events, wheat harvesting operations usually start when the grain moisture content is about 12–16%."
 grain moisture loss depends on air temperature, humidity, and crop cultivar -->

- [Sorghum](https://www.uaex.uada.edu/publications/pdf/mp297/gschapter10.pdf) : "Once harvested, grain sorghum should quickly be dried down to a MC of about 12% wet basis for proper storage, particularly if it is going to be stored for several months"
<!-- "In Arkansas, grain sorghum does not always field-dry to a safe moisture content (MC) of 12% wet basis due to our humid environment."
Acc to their table 10-1, can range from 9.6 (if 40 degrees & 30% relative humidity) to 17.2. -->


In [31]:
wheat_pct_water = .11 # placeholder...
millet_pct_water = .11 # placeholder...
sorg_pct_water = .11 # placeholder...


kg_sorg = ALL_sorg * 1000  # total sorghum in kg
kg_sorg_DM = kg_sorg * (1-sorg_pct_water)

kg_wheat = ALL_wheat * 1000  # wheat to kg
kg_wheat_DM = kg_wheat * (1-wheat_pct_water)

kg_millet = ALL_millet * 1000  # millet to kg
kg_millet_DM = kg_millet * (1-millet_pct_water)

We can now use the dry matter conversion to calculate calories:

In [32]:
# Each kg has 10 100-gram portions
sorg_kcal = kg_sorg_DM * 10 * sorg_conv
wheat_kcal = kg_wheat_DM * 10 * wheat_conv
millet_kcal = kg_millet_DM * 10 * millet_conv


print(f"Sorghum calories: {sorg_kcal:.2f} Kcal")
print(f"Wheat calories: {wheat_kcal:.2f} Kcal")
print(f"Millet calories: {millet_kcal:.2f} Kcal")

total_kcal = sorg_kcal + wheat_kcal + millet_kcal
print(f"Total calories from all grains: {total_kcal:.2f} Kcal")


Sorghum calories: 9757204436778.40 Kcal
Wheat calories: 3804467725357.20 Kcal
Millet calories: 2983373206371.40 Kcal
Total calories from all grains: 16545045368507.00 Kcal


## Include non-grain sources of food

It is difficult to get data on other food sources. Instead, we will just assume that 70% of calories are from grain (following Clingendael "From hunger to death" report p.3, which in turn cites FEWSNET).


In [34]:
# Add additional calories from non-grain sources

pct_non_grain_kcal = .30
grain_kcal_percentage = 1 - pct_non_grain_kcal  # 70%

total_kcal_with_non_grain = total_kcal / grain_kcal_percentage

non_grain_kcal = total_kcal_with_non_grain - total_kcal

print(f"Calories from non-grain sources: {non_grain_kcal:.2f} Kcal")

print(f"Total calories including grain and non-grain sources: {total_kcal_with_non_grain:.2f} Kcal")

# (verify no errors) non_grain_kcal / (total_kcal + non_grain_kcal) # should be same as pct_non_grain_kcal - correct.


Calories from non-grain sources: 7090733729360.14 Kcal
Total calories including grain and non-grain sources: 23635779097867.14 Kcal


## Sanity check

I have no idea how many calories that is! For now, just to get a rough sense, we will assume each human consumes 2200 kCal per day and ask how many days of food this would provide for 50 million people, a rough estimate of the population of Sudan.



In [35]:
# calculate how many days would the calories would be sufficient to feed 50M people.

caloric_requirement_per_person = 2200  # kcal/day
population = 5e7  # 50 million people

daily_caloric_needs = population * caloric_requirement_per_person

days_of_food = total_kcal_with_non_grain / daily_caloric_needs

print(f"The total calories would feed {population:,} people for approximately {days_of_food:.2f} days.")

The total calories would feed 50,000,000.0 people for approximately 214.87 days.


In [36]:
# If instead the population is only 45 million and we assume 2000 kcal / day:

caloric_requirement_per_person = 2000  # kcal/day
population = 45e6  # 45 million people

daily_caloric_needs = population * caloric_requirement_per_person

days_of_food = total_kcal_with_non_grain / daily_caloric_needs

print(f"The total calories would feed {population:,} people for approximately {days_of_food:.2f} days.")

The total calories would feed 45,000,000.0 people for approximately 262.62 days.


## Next steps

This was just a first step. It should make clear there are a number of assumptions along the way that need to be investigated more thoroughly. Next, in another notebook we will try to consider the impact of the limited supply of calories on excess death estimates, making assumptions about unequal distributions of food among the population and BMI estimates for these different segments.